In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

def compile_csv(startYear, endYear):
    df_list = []
    for i in range(startYear, endYear + 1):
        off_filename = str(i) + 'Offense.csv'
        def_filename = str(i) + 'Defense.csv'
        off_df = pd.read_csv(off_filename)
        off_df["Unit"] = ["Offense" if pos != "LS" else "Special" for pos in off_df["Pos"]]
        def_df = pd.read_csv(def_filename)
        def_df["Unit"] = ["Defense" if (pos != "K" and pos != "P") else "Special" for pos in def_df["Pos"]]
        df_list.append(off_df)
        df_list.append(def_df)
    data = pd.concat(df_list)
    return data

df = compile_csv(2000, 2017)
df["Player"] = [x.split("\\")[0] for x in df["Player"]]

# Parse out Drafted (tm/rnd/yr) column
df["Drafted (tm/rnd/yr)"] = df["Drafted (tm/rnd/yr)"].where(pd.notnull(df["Drafted (tm/rnd/yr)"]), None)
df["DraftTeam"] = [x.split(" / ")[0] if x != None else None for x in df["Drafted (tm/rnd/yr)"]]
df["DraftRd"] = [x.split(" / ")[1] if x != None else None for x in df["Drafted (tm/rnd/yr)"]]
df["DraftRd"] = df["DraftRd"].str.replace('[a-zA-Z]+', '')
df["DraftPick"] = [x.split(" / ")[2] if x != None else None for x in df["Drafted (tm/rnd/yr)"]]
df["DraftPick"] = df["DraftPick"].str.replace('[a-zA-Z_]+', '')
df = df.drop(["Drafted (tm/rnd/yr)"], axis=1)

# Convert height to inches
def convert_height(x):
    feet = x.split("-")[0]
    inches = x.split("-")[1]
    height = (int(feet) * 12) + int(inches)
    return height
df['Height'] = df['Height'].apply(convert_height)

df.describe()

,Rk,Year,AV,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle
count,5636.000000,5636.000000,4089.000000,5636.000000,5636.000000,5553.000000,4413.000000,3828.000000,4374.000000,3686.000000,3742.000000
mean,78.977999,2008.529631,16.823918,73.763840,242.491306,4.770501,32.966010,20.840909,114.210791,7.270757,4.381168
std,45.539166,5.180094,23.971114,2.648355,44.600760,0.300133,4.163298,6.347158,9.146190,0.419310,0.268175
min,1.000000,2000.000000,-4.000000,65.000000,149.000000,4.220000,17.500000,2.000000,74.000000,6.340000,3.730000
25%,40.000000,2004.000000,1.000000,72.000000,206.000000,4.540000,30.000000,16.000000,109.000000,6.960000,4.190000
50%,79.000000,2009.000000,7.000000,74.000000,233.000000,4.690000,33.000000,21.000000,115.000000,7.180000,4.330000
75%,118.000000,2013.000000,23.000000,76.000000,273.000000,4.950000,36.000000,25.000000,121.000000,7.510000,4.540000
max,172.000000,2017.000000,255.000000,82.000000,375.000000,6.050000,46.000000,49.000000,147.000000,9.120000,5.560000


In [2]:
df2 = df[['Year','Pos', 'Height', 'Wt', '40YD', 'Vertical', 'BenchReps', 'Broad Jump', '3Cone', 'Shuttle','Unit', 'DraftRd']]
df2['DraftRd'] = df2['DraftRd'].fillna(0)

df2 = df2.fillna('-1')
df2 = df2.apply(pd.to_numeric, errors='ignore')
rank_40 = []
rank_vert = []
rank_bench = []
rank_broad = []
rank_3cone = []
rank_shuttle = []
for index, row in df2.iterrows():
    year = row['Year']
    position = row['Pos']
    #Calculate 40YD quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['40YD'] != -1)]["40YD"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['40YD'] != -1)]["40YD"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['40YD'] != -1)]["40YD"].quantile(.75)
    quartile = 4
    if row['40YD'] == -1:
        quartile = 5
    elif row['40YD'] <= quartile_1:
        quartile = 1
    elif row['40YD'] <= median:
        quartile = 2
    elif row['40YD'] <= quartile_3:
        quartile = 3
    rank_40.append(quartile)
    
    #Calculate vert quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Vertical'] != -1)]["Vertical"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Vertical'] != -1)]["Vertical"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Vertical'] != -1)]["Vertical"].quantile(.75)
    quartile = 4
    if row['Vertical'] == -1:
        quartile = 5
    elif row['Vertical'] >= quartile_3:
        quartile = 1
    elif row['Vertical'] >= median:
        quartile = 2
    elif row['Vertical'] >= quartile_1:
        quartile = 3
    rank_vert.append(quartile)
    
    #Calculate BenchReps quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['BenchReps'] != -1)]["BenchReps"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['BenchReps'] != -1)]["BenchReps"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['BenchReps'] != -1)]["BenchReps"].quantile(.75)
    quartile = 4
    if row['BenchReps'] == -1:
        quartile = 5
    elif row['BenchReps'] >= quartile_3:
        quartile = 1
    elif row['BenchReps'] >= median:
        quartile = 2
    elif row['BenchReps'] >= quartile_1:
        quartile = 3
    rank_bench.append(quartile)
    
    #Calculate Broad Jump quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Broad Jump'] != -1)]["Broad Jump"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Broad Jump'] != -1)]["Broad Jump"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Broad Jump'] != -1)]["Broad Jump"].quantile(.75)
    quartile = 4
    if row['Broad Jump'] == -1:
        quartile = 5
    elif row['Broad Jump'] >= quartile_3:
        quartile = 1
    elif row['Broad Jump'] >= median:
        quartile = 2
    elif row['Broad Jump'] >= quartile_1:
        quartile = 3
    rank_broad.append(quartile)
    
    #Calculate 3Cone quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['3Cone'] != -1)]["3Cone"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['3Cone'] != -1)]["3Cone"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['3Cone'] != -1)]["3Cone"].quantile(.75)
    quartile = 4
    if row['3Cone'] == -1:
        quartile = 5
    elif row['3Cone'] <= quartile_1:
        quartile = 1
    elif row['3Cone'] <= median:
        quartile = 2
    elif row['3Cone'] <= quartile_3:
        quartile = 3
    rank_3cone.append(quartile)
    
    #Calculate Shuttle quartile
    quartile_1 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Shuttle'] != -1)]["Shuttle"].quantile(.25)
    median = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Shuttle'] != -1)]["Shuttle"].quantile(.5)
    quartile_3 = df2[(df2['Year'] == year) & (df2['Pos'] == position) & (df2['Shuttle'] != -1)]["Shuttle"].quantile(.75)
    quartile = 4
    if row['Shuttle'] == -1:
        quartile = 5
    elif row['Shuttle'] <= quartile_1:
        quartile = 1
    elif row['Shuttle'] <= median:
        quartile = 2
    elif row['Shuttle'] <= quartile_3:
        quartile = 3
    rank_shuttle.append(quartile)
df2['40_quartile_yr_pos'] = rank_40 
df2['vert_quartile_yr_pos'] = rank_vert
df2['bench_quartile_yr_pos'] = rank_bench
df2['broad_quartile_yr_pos'] = rank_broad
df2['3cone_quartile_yr_pos'] = rank_3cone
df2['shuttle_quartile_yr_pos'] = rank_shuttle
df2

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Year,Pos,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Unit,DraftRd,40_quartile_yr_pos,vert_quartile_yr_pos,bench_quartile_yr_pos,broad_quartile_yr_pos,3cone_quartile_yr_pos,shuttle_quartile_yr_pos
0,2000,WR,75,191,4.53,33.0,-1.0,130.0,7.09,4.18,Offense,0,2,3,5,1,3,3
1,2000,QB,75,229,4.91,34.0,-1.0,108.0,7.71,4.59,Offense,6,2,1,5,2,4,4
2,2000,WR,71,180,4.59,36.0,-1.0,123.0,7.22,4.16,Offense,6,3,2,5,1,4,2
3,2000,OT,76,332,5.26,29.0,28.0,97.0,7.78,4.72,Offense,2,2,1,1,3,2,1
4,2000,RB,71,193,4.50,-1.0,-1.0,-1.0,-1.00,-1.00,Offense,5,1,5,5,5,5,5
5,2000,WR,73,218,4.62,37.5,-1.0,124.0,6.91,4.09,Offense,3,4,1,5,1,1,1
6,2000,TE,76,254,4.66,32.5,-1.0,121.0,6.97,4.18,Offense,5,1,2,5,1,2,2
7,2000,TE,74,228,4.78,31.0,-1.0,110.0,7.42,4.22,Offense,5,2,3,5,3,4,2
8,2000,WR,71,194,4.58,-1.0,-1.0,-1.0,-1.00,-1.00,Offense,1,3,5,5,5,5,5
9,2000,WR,66,171,4.55,32.5,-1.0,106.0,6.61,3.84,Offense,5,2,4,5,4,1,1


In [4]:
y = df2['DraftRd']
X = df2.drop(['DraftRd', 'Year'], axis=1)
X_encoded = pd.get_dummies(X, columns=['Pos', 'Unit'])
X_encoded

,Height,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,40_quartile_yr_pos,vert_quartile_yr_pos,...,Pos_OT,Pos_P,Pos_QB,Pos_RB,Pos_SS,Pos_TE,Pos_WR,Unit_Defense,Unit_Offense,Unit_Special
0,75,191,4.53,33.0,-1.0,130.0,7.09,4.18,2,3,...,0,0,0,0,0,0,1,0,1,0
1,75,229,4.91,34.0,-1.0,108.0,7.71,4.59,2,1,...,0,0,1,0,0,0,0,0,1,0
2,71,180,4.59,36.0,-1.0,123.0,7.22,4.16,3,2,...,0,0,0,0,0,0,1,0,1,0
3,76,332,5.26,29.0,28.0,97.0,7.78,4.72,2,1,...,1,0,0,0,0,0,0,0,1,0
4,71,193,4.50,-1.0,-1.0,-1.0,-1.00,-1.00,1,5,...,0,0,0,1,0,0,0,0,1,0
5,73,218,4.62,37.5,-1.0,124.0,6.91,4.09,4,1,...,0,0,0,0,0,0,1,0,1,0
6,76,254,4.66,32.5,-1.0,121.0,6.97,4.18,1,2,...,0,0,0,0,0,1,0,0,1,0
7,74,228,4.78,31.0,-1.0,110.0,7.42,4.22,2,3,...,0,0,0,0,0,1,0,0,1,0
8,71,194,4.58,-1.0,-1.0,-1.0,-1.00,-1.00,3,5,...,0,0,0,0,0,0,1,0,1,0
9,66,171,4.55,32.5,-1.0,106.0,6.61,3.84,2,4,...,0,0,0,0,0,0,1,0,1,0


In [5]:
clf = RandomForestClassifier()
grid_values = {"max_depth": [2, 8, 10, 20], "n_estimators": [10, 20, 50], "max_features": [5, 10, 20]}
grid_rf = GridSearchCV(clf, param_grid = grid_values, cv=5)
grid_rf.fit(X_encoded, y)
print("Grid Search Complete: {}, {}".format(grid_rf.best_score_, grid_rf.best_params_))

Grid Search Complete: 0.3757984386089425, {'max_depth': 8, 'max_features': 10, 'n_estimators': 50}


In [6]:
feature_importances = pd.Series(grid_rf.best_estimator_.feature_importances_, index=X_encoded.columns)
feature_importances.sort_values(ascending=False)

40_quartile_yr_pos         0.137047
40YD                       0.127277
Wt                         0.117020
3Cone                      0.068157
BenchReps                  0.067429
Shuttle                    0.063070
Broad Jump                 0.062779
Vertical                   0.057784
Height                     0.052130
broad_quartile_yr_pos      0.030232
vert_quartile_yr_pos       0.029199
shuttle_quartile_yr_pos    0.029168
bench_quartile_yr_pos      0.026396
3cone_quartile_yr_pos      0.026066
Unit_Defense               0.014347
Unit_Special               0.013347
Unit_Offense               0.009328
Pos_P                      0.007060
Pos_WR                     0.006928
Pos_DT                     0.006813
Pos_OLB                    0.005751
Pos_FS                     0.005033
Pos_QB                     0.005026
Pos_CB                     0.004748
Pos_OG                     0.004390
Pos_FB                     0.003825
Pos_SS                     0.003763
Pos_ILB                    0

# Split between skill
Three splits: Speed Skill (CB, SS, FS, WR, RB), Mid Skill (OLB, ILB, FB, TE, QB, P) and Big Skill (DT, OT, OG, DE, LS)

In [7]:
skill = {'speed_skill': ['CB', 'SS', 'FS', 'WR', 'RB'], 'mid_skill': ['OLB', 'ILB', 'FB', 'TE', 'QB', 'P'], 'big_skill': ['DT', 'OT', 'OG', 'DE', 'LS']}

for key, value in skill.items():
    df_skill = df2[df2['Pos'].isin(value)]
    y = df_skill['DraftRd']
    X_encoded = pd.get_dummies(df_skill.drop(['DraftRd'], axis=1), columns=['Pos', 'Unit'])
    
    clf = RandomForestClassifier()
    grid_values = {"max_depth": [4, 6, 8, 10], "n_estimators": [10, 50, 100]}
    grid_skill = GridSearchCV(clf, param_grid = grid_values, cv=5)
    grid_skill.fit(X_encoded, y)
    print("------------\n{}\nScore: {}\nParameters: {}\nFeatures: {}".format(key, grid_skill.best_score_, grid_skill.best_params_, pd.Series(grid_skill.best_estimator_.feature_importances_, X_encoded.columns).sort_values(ascending=False)))


------------
speed_skill
Score: 0.38759031024224394
Parameters: {'max_depth': 6, 'n_estimators': 50}
Features: 40YD                       0.199474
40_quartile_yr_pos         0.157968
3Cone                      0.081430
Wt                         0.079538
Broad Jump                 0.065416
Vertical                   0.058186
Shuttle                    0.056897
Year                       0.045077
BenchReps                  0.036366
Height                     0.033910
vert_quartile_yr_pos       0.028907
3cone_quartile_yr_pos      0.027721
broad_quartile_yr_pos      0.024790
shuttle_quartile_yr_pos    0.022211
bench_quartile_yr_pos      0.021239
Unit_Defense               0.014322
Pos_CB                     0.010655
Pos_WR                     0.008386
Unit_Offense               0.007240
Pos_RB                     0.006932
Pos_SS                     0.006760
Pos_FS                     0.006573
dtype: float64
------------
mid_skill
Score: 0.389819587628866
Parameters: {'max_depth': 6, 'n_es

In [8]:
# Try with a k-nearest neighbors classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler

for key, value in skill.items():
    df_skill = df2[df2['Pos'].isin(value)]
    y = df_skill['DraftRd']
    X_encoded = pd.get_dummies(df_skill.drop(['DraftRd'], axis=1), columns=['Pos', 'Unit'])
    scaler = MinMaxScaler()
#     scaler = StandardScaler()
    X_scaled_encoded = scaler.fit_transform(X_encoded)
    X_df = pd.DataFrame(X_scaled_encoded, columns=X_encoded.columns)
#     print(X_df)
    
    clf = KNeighborsClassifier(n_jobs=-1)
    grid_values = {"n_neighbors": [2, 5, 10, 20, 50, 100, 200]}
    grid_skill = GridSearchCV(clf, param_grid = grid_values, cv=5)
    grid_skill.fit(X_df, y)
    print("------------\n{}\nScore: {}\nParameters: {}\n".format(key, grid_skill.best_score_, grid_skill.best_params_))


------------
speed_skill
Score: 0.3671908202294943
Parameters: {'n_neighbors': 100}

------------
mid_skill
Score: 0.38079896907216493
Parameters: {'n_neighbors': 100}

------------
big_skill
Score: 0.3483535528596187
Parameters: {'n_neighbors': 50}



In [9]:
# Try with logistic regression classifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

for key, value in skill.items():
    df_skill = df2[df2['Pos'].isin(value)]
    y = df_skill['DraftRd']
    X_encoded = pd.get_dummies(df_skill.drop(['DraftRd'], axis=1), columns=['Pos', 'Unit'])
    scaler = MinMaxScaler()
#     scaler = StandardScaler()
    X_scaled_encoded = scaler.fit_transform(X_encoded)
    X_df = pd.DataFrame(X_scaled_encoded, columns=X_encoded.columns)
#     print(X_df)
    
    clf = LogisticRegression(penalty='l2')
    grid_values = {"C": [0.001, 0.01, 0.1, .5, 1, 5, 10, 50]}
    grid_skill = GridSearchCV(clf, param_grid = grid_values, cv=5)
    grid_skill.fit(X_df, y)
    print("------------\n{}\nScore: {}\nParameters: {}\n".format(key, grid_skill.best_score_, grid_skill.best_params_))

------------
speed_skill
Score: 0.37909052273693156
Parameters: {'C': 0.5}

------------
mid_skill
Score: 0.39239690721649484
Parameters: {'C': 1}

------------
big_skill
Score: 0.3558636626227614
Parameters: {'C': 1}



# Try a gradient boosted tree with xgboost
Train an xgboost model and tune parameters for best result

In [14]:
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

y = df2['DraftRd']
X == df2.drop(['DraftRd', 'Year'], axis=1)
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train_scaled = scaler.fit_transform(pd.get_dummies(X_train, columns=['Pos', 'Unit']))
X_test_scaled = scaler.transform(pd.get_dummies(X_test, columns=['Pos', 'Unit']))

In [35]:
xgbClf = xgb.XGBClassifier(n_estimators = 250, max_depth = 25, learning_rate = 0.1, gamma = 0)
xgbClf.fit(X_train_scaled, y_train)
y_pred = xgbClf.predict(X_test_scaled)

print(classification_report(y_test, y_pred))
print("Accuracy: {}\nConfusion Matrix: {}".format(accuracy_score(y_test, y_pred), confusion_matrix(y_test,y_pred)))

             precision    recall  f1-score   support

          0       0.47      0.71      0.57       510
          1       0.31      0.36      0.34       146
          2       0.10      0.09      0.09       120
          3       0.10      0.07      0.09       149
          4       0.08      0.06      0.07       136
          5       0.11      0.05      0.07       125
          6       0.05      0.03      0.03       113
          7       0.05      0.02      0.03       110

avg / total       0.25      0.32      0.27      1409

Accuracy: 0.32292405961674947
Confusion Matrix: [[361  26  17  26  25  19  20  16]
 [ 36  53  24  11  10   1   6   5]
 [ 47  21  11  11  10   8  11   1]
 [ 58  32  16  11  15   9   5   3]
 [ 62   9  18  19   8   7   6   7]
 [ 68   7  17   9   9   6   5   4]
 [ 67  11   5  13  10   3   3   1]
 [ 61  11   7   9  11   4   5   2]]


# Conclusions
## No clear relationships correlating to round were found when exploring
I wasn't able to find any real clear indicators in the physical combine results which were indicative of a player being drafted in a certain round.  This isn't all that surprising as there are many other factors other than a player's ability to test well that go into being drafted.  There also is I think a large discrepancy in the data for undrafted players vs. players in each round, making it hard to predict a player's draft round.  One possible alternative test I may do in the future is to just try and predict the binary drafted or undrafted.  There are also many player's who forego various events, and handling for null values in this dataset is difficult as removing all players who skipped at least one event would have left a very skimpy dataset.

# Next Steps
- Gather college stats for previous season as potentially more important features
- Split the model out by each position
- Explore for other data sets which might provide more insight into where a player will be drafted, such as mock-drafts